# Part 3: TF-IDF to understand genres and communities
The questions below are based on Lecture 7, part 2, 4, 5, 6 (and a little bit on part 3).

## Explain the concept of TF-IDF in your own words and how it can help you understand the genres and communities

TF-IDF is a way to score/measure how important a word is for a text in relation to how frequent the word is used overall in all texts. For instance, if a word appears a lot in a text, it has a high TF, but it can still have a low IDF if it appears frequently in a lot of texts like for instance, we can imagine words like "the", "and", "good", "to" etc. word have high TF but low IDF.

TF is the frequency of the term in the text.
IDF is how rare the word is across all texts.

So in summary TF-IDF gives higher scores to words that are frequent in one document but rare across others.

This score/measure can help us understand the genres, because we can look through the texts of all bands in each genre. This will then highlight genre-specific terms/words. This could make it easier to cluster similar artists, as artist with similar high-scoring TF-IDF words can be grouped into comunities or genre clusters. TF-IDF can also help distinguish what makes each genre unique, by boosting words/terms that are frequent in one genre (high TF) and then downweighing words that are commen across all genres (low IDF). 

## Calculate and visualize TF-IDF for the genres and communities.

## Use the matrix D(Lecture 7, part 2) to dicusss the difference between the word-clouds between genres and communities.